# Predicting Pipe Failure Conditions with Decision Trees and Random Forests

In [12]:
#dependancies list
'''
- pymssql
- pandas
- json


'''

'\n- pymssql\n- pandas\n- json\n\n\n'

## Importing Data from SQL

In [13]:
import pandas as pd
import json
import pypyodbc

conn = pypyodbc.connect("DRIVER={SQL Server};\
                        SERVER=yvwchallenge.chqn5g6ohkqb.ap-southeast-2.rds.amazonaws.com;\
                        UID=admin;\
                        PWD=passwordpassword;\
                        DATABASE=yvwchallenge")
#data["title"] = data.boilerplate.map(lambda x: json.loads(x).get("title", ""))
#data["body"] = data.boilerplate.map(lambda x: json.loads(x).get("body", ""))
#data = pd.read_sql(data_sql)

yvw_df = pd.read_sql('SELECT * FROM yvwdataframe_firstbreak', conn)
yvw_df.head()

,asset id,count,event time,event date,class structure,class structure id,material,nominal size (mm),soil class,soil description,...,failure description,job plan,job plan number,service location id,formatted address,city,post code,latitude,longitude,months_break
0,2134909,1,1:26:00 AM,11/7/2011,Repair Burst Water Main,WRBRST,CAST IRON CEMENT LINED,100,EX,Moderate shrink/swell potential.,...,UNKN,None,None,1271238,1 BAROSSA AVENUE VERMONT SOUTH 3133,VERMONT SOUTH,3133,-37.85526433,145.19084730,152
1,2134910,1,10:31:00 AM,1/18/2010,Repair Burst Water Main,WRBRST,CAST IRON CEMENT LINED,100,EX,Moderate shrink/swell potential.,...,UNKN,None,None,1271236,5 BAROSSA AVENUE VERMONT SOUTH 3133,VERMONT SOUTH,3133,-37.85532502,145.19127780,145
2,2134913,1,4:12:00 PM,3/16/2006,Repair Burst Water Main,WRBRST,UPVC-THICK WALLED PRESSURE PIPE(INCL BLUE BRUTE),100,EX,Moderate shrink/swell potential.,...,UNKN,None,None,1523784,13 ADELE STREET VERMONT 3133,VERMONT,3133,-37.83213171,145.20622920,19
3,2134915,1,11:54:00 AM,11/13/1998,Repair Burst Water Main,WRBRST,CAST IRON CEMENT LINED,100,EX,Moderate shrink/swell potential.,...,UNKN,None,None,1265242,8 CALDWELL ROAD VERMONT 3133,VERMONT,3133,-37.83383109,145.21071050,157
4,2134918,1,1:00:00 AM,2/16/1997,Repair Burst Water Main,WRBRST,CAST IRON CEMENT LINED,100,EX,Moderate shrink/swell potential.,...,UNKN,None,None,1265573,2 AUBREY STREET VERMONT 3133,VERMONT,3133,-37.83534315,145.20643560,149


In [14]:
yvw_df[["failure description"]].count()

failure description    31365
dtype: int64

In [15]:
yvw_df.to_pickle("yvw_df_firstbreak.pkl")

In [16]:
yvw_df = pd.read_pickle("yvw_df_firstbreak.pkl")
yvw_df = pd.DataFrame(yvw_df)

In [17]:
print(yvw_df.groupby(["description"])[["asset id"]].count())
print(yvw_df.groupby(["nominal size (mm)"])[["asset id"]].count())
print(yvw_df["failure description"].count())


                                                    asset id
description                                                 
Fault,Water (Drinking And Ndw) Works,Hydrant-fi...         1
Fault,Water (Drinking And Ndw) Works,Hydrant-fi...         2
Fault,Water (Drinking And Ndw) Works,Hydrant-fi...         1
Fault,Water (Drinking And Ndw) Works,Hydrant-fi...         1
Fault,Water (Drinking And Ndw) Works,Hydrant-fi...         2
Fault,Water (Drinking And Ndw) Works,Hydrant-fi...         1
Fault,Water (Drinking And Ndw) Works,Hydrant-fi...         1
Fault,Water (Drinking And Ndw) Works,Hydrant-fi...         1
Fault,Water (Drinking And Ndw) Works,Main,Drive...         1
Fault,Water (Drinking And Ndw) Works,Main,Drive...         2
Fault,Water (Drinking And Ndw) Works,Main,Drive...         2
Fault,Water (Drinking And Ndw) Works,Main,Drive...         1
Fault,Water (Drinking And Ndw) Works,Main,Drive...         1
Fault,Water (Drinking And Ndw) Works,Main,Footp...         1
Fault,Water (Drinking An

### decision tree model

In [18]:
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import cross_val_score
from collections import defaultdict
d = defaultdict(LabelEncoder)

fit = yvw_df.apply(lambda x: d[x.name].fit_transform(x))
yvw_df = yvw_df.reindex()
#yvw_df.apply(LabelEncoder().fit_transform)

X = fit.dropna()
y = X['months_break']
X.drop('months_break', axis = 1, inplace = True)  

fit.head()

,asset id,count,event time,event date,class structure,class structure id,material,nominal size (mm),soil class,soil description,...,failure description,job plan,job plan number,service location id,formatted address,city,post code,latitude,longitude,months_break
0,12858,0,412,1603,0,0,1,2,1,2,...,1,0,0,14971,85,167,67,24508,22938,179
1,12859,0,62,174,0,0,1,2,1,2,...,1,0,0,14970,20648,167,67,24514,22959,172
2,12860,0,744,2860,0,0,18,2,1,2,...,1,0,0,25865,5080,166,67,22327,23572,46
3,12861,0,228,1174,0,0,1,2,1,2,...,1,0,0,14579,25885,166,67,22542,23713,184
4,12862,0,360,2342,0,0,1,2,1,2,...,1,0,0,14608,9317,166,67,22707,23580,176


In [19]:
 # Fits the model
model = DecisionTreeClassifier()
model.fit(X, y)

# Helper function to visualise Decision Trees (creates a file tree.png)
from sklearn.tree import export_graphviz
from os import system

def build_tree_image(model):
    dotfile = open("tree.dot", "w")
    export_graphviz(model,
                    out_file = dotfile,
                    feature_names = X.columns)
    dotfile.close()
    system("dot -Tpng tree.dot -o tree.png")
build_tree_image(model)
    
model
scores = cross_val_score(model, X, y, cv = 40)
print("CV AUC {}, Average AUC {}",scores, scores.mean(), scores.max())
#20 columns

('CV AUC {}, Average AUC {}', array([ 0.071949  ,  0.08380952,  0.06439024,  0.08075773,  0.0795569 ,
        0.07165109,  0.09013786,  0.08172043,  0.0863388 ,  0.0972067 ,
        0.09728507,  0.08955224,  0.09490741,  0.08343126,  0.05608592,
        0.10935601,  0.08353808,  0.082397  ,  0.08238276,  0.09266409,
        0.08769634,  0.07793923,  0.09785523,  0.07744565,  0.07407407,
        0.07520891,  0.09659091,  0.08393632,  0.09809663,  0.0880597 ,
        0.07564297,  0.06687403,  0.06803797,  0.11935484,  0.06546645,
        0.075     ,  0.08717949,  0.07986111,  0.07637655,  0.06921676]), 0.082975782201272039, 0.11935483870967742)


In [20]:
print(yvw_df.columns.tolist())

['asset id', 'count', 'event time', 'event date', 'class structure', 'class structure id', 'material', 'nominal size (mm)', 'soil class', 'soil description', 'grant description', 'pipe inside diameter (mm)', 'pipe material', 'pipe shape', 'construction date (yyyymmdd)', 'pipe lining', 'date insulated', 'tapping status', 'road name', 'road type', 'pipe length (m)', 'pipe status', 'height start of pipe (meters above sea level)', 'height end of pipe (meters above sea level)', 'latitude start of pipe', 'lattitude end of pipe', 'longitude start of pipe', 'longitude end of pipe', 'distribution zone id', 'water quality zone id', 'description', 'failure code', 'failure description', 'job plan', 'job plan number', 'service location id', 'formatted address', 'city', 'post code', 'latitude', 'longitude', 'months_break']


In [21]:
# Random Forest Classification
import pandas
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score

seed = 7
num_trees = 200
max_features = 15
kfold = model_selection.KFold(n_splits=20, random_state=seed)
model = RandomForestClassifier(n_estimators=num_trees, max_features=max_features, n_jobs=-1)
results = model_selection.cross_val_score(model, X, y, cv=kfold)
print(results.mean())

0.0740951112759


In [22]:
#model = DecisionTreeClassifier(max_depth = 2, min_samples_leaf = 5)
#model.fit(X, y)
#build_tree_image(model)

###  random forest model 

In [23]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators = 100, n_jobs=-1)    
model.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

### Extracting importance of features

In [24]:
features = X.columns
feature_importances = model.feature_importances_

features_df = pd.DataFrame({"Features": features, "Importance Score": feature_importances})
features_df.sort_values("Importance Score", inplace = True, ascending = False)

features_df.head()
print(features_df)

                                         Features  Importance Score
14                   construction date (yyyymmdd)          0.064576
3                                      event date          0.054524
20                                pipe length (m)          0.052020
2                                      event time          0.051354
0                                        asset id          0.050604
35                            service location id          0.049958
36                              formatted address          0.049094
18                                      road name          0.047756
23    height end of pipe (meters above sea level)          0.047447
22  height start of pipe (meters above sea level)          0.047086
25                          lattitude end of pipe          0.044273
24                         latitude start of pipe          0.044080
26                        longitude start of pipe          0.043902
27                          longitude end of pip

### Eval the Random Forest model using cross-validation

In [25]:
from sklearn.cross_validation import cross_val_score

scores = cross_val_score(model, X, y, scoring = "roc_auc")
print("CV AUC {}, Average AUC {}".format(scores, scores.mean()))

for n_trees in range(1, 100, 10):
    model = RandomForestClassifier(n_estimators = n_trees)
    scores = cross_val_score(model, X, y)
    print("n trees: {}, CV AUC {}, Average AUC {}", n_trees, scores, scores.mean())

c:\programdata\miniconda3\envs\yvw\lib\site-packages\sklearn\cross_validation.py:553: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=3.
  % (min_labels, self.n_folds)), Warning)


ValueError: multiclass format is not supported

In [ ]:
## 1. Building a model with more relevant features

model = RandomForestClassifier(n_estimators = 50)

# Continue to add features to X
#     Build dummy features, include quantitative features, or add text features
X = data[["image_ratio", "html_ratio", "recipe", "label"]].dropna()
y = X["label"]
X.drop("label", axis = 1, inplace = True)

## 2a. Evaluate predictive performance for the given feature set
scores = cross_val_score(model, X, y, scoring = "roc_auc")
print("CV AUC {}, Average AUC {}".format(scores, scores.mean()))

# 3 (BONUS): Adding in text features

# Check for keywords in the title
data["PhotoInTitle"] = data["title"].fillna("").str.lower().str.contains("photo").astype(int)
X = data[["image_ratio", "html_ratio", "recipe", "PhotoInTitle", "label"]].dropna()
X.drop("label", axis = 1, inplace = True)

scores = cross_val_score(model, X, y, scoring = "roc_auc")
print("CV AUC {}, Average AUC {}".format(scores, scores.mean()))

## 2b. Evaluating feature importances

# Fit a model on the whole dataset
model.fit(X, y)

# Get columns and their scores
features = X.columns
feature_importances = model.feature_importances_
features_df = pd.DataFrame({"Features": features, "Importance Score": feature_importances})
features_df.sort_values("Importance Score", inplace = True, ascending = False)

features_df.head()